In [1]:
%pip install langchain_community langchain_text_splitters langchain_openai langchain_chroma gradio python-dotenv pypdf

# New Section

## Requirements
#### Vector Database = langchain_chroma
#### Embeddings Model Provider= SentenceTransformerEmbeddings
#### Embedding Model Name = model_name="all-mpnet-base-v2"

In [ ]:
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')

# # Define the Google Drive path where your documents are stored
# DATA_PATH = '/content/drive/MyDrive/local_rag/data'

In [ ]:
DATA_PATH = r"data"
CHROMA_PATH = r"chroma_db"

In [ ]:
import os

# Get a list of all files in the specified folder
file_list = os.listdir(DATA_PATH)

# Filter the list to only include files with .pdf or .docx extensions
pdf_files = [f for f in file_list if f.endswith('.pdf')]
docx_files = [f for f in file_list if f.endswith('.docx')]

# Example: Print the list of PDF and DOCX files in the folder
print("PDF Files:", pdf_files)
print("DOCX Files:", docx_files)

PDF Files: ['AttentionIsAllyouNeed.pdf', 'AI Bootcamp Syllabus 2024-1.pdf']
DOCX Files: []


In [ ]:
import os
current_directory = os.getcwd()
print(current_directory)

/Users/avineetsharma/Project/Doc-Chat


In [ ]:
# import the .env file
from dotenv import load_dotenv
load_dotenv()



True

In [ ]:
import os
import time
from uuid import uuid4
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

embeddings_model = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings_model,
    persist_directory=CHROMA_PATH,
)


/var/folders/5p/y9vfdxz132b7tj62wggjh0_m0000gn/T/ipykernel_13382/3877657826.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")
/opt/anaconda3/envs/dev/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/5p/y9vfdxz132b7tj62wggjh0_m0000gn/T/ipykernel_13382/3877657826.py:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An

In [ ]:
def indexed_files(vector_store):
    stored_files = set(
        [
            meta["source"].split("/")[-1]
            for meta in vector_store._collection.get(include=["metadatas"])["metadatas"]
            if meta and "source" in meta
        ]
    )

    print ("stored_files:",stored_files)
    return stored_files

indexed_files(vector_store)

stored_files: set()


set()

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models import ChatCohere

# Initialize Cohere LLM
llm = ChatCohere(model="command-r-plus-08-2024")

def summarize_text(text):
    """Generate a summary using an LLM"""
    summarize_chain = load_summarize_chain(llm, chain_type="map_reduce")
    summary = summarize_chain.run([text])
    return summary

/var/folders/5p/y9vfdxz132b7tj62wggjh0_m0000gn/T/ipykernel_13382/2873985129.py:5: LangChainDeprecationWarning: The class `ChatCohere` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import ChatCohere``.
  llm = ChatCohere(model="command-r-plus-08-2024")


In [ ]:

def process_files(vector_store, embeddings_model):
    """
    Processes files in DATA_PATH, adding new files and removing deleted files.
    """
    current_files = set(os.listdir(DATA_PATH))
    updates = False
    print ("current_files:",current_files)

    stored_files = indexed_files(vector_store)


    # Find new files
    new_files = current_files - stored_files
    for filename in new_files:
        updates = True
        if filename.endswith(".pdf"):
            filepath = os.path.join(DATA_PATH, filename)
            loader = PyPDFLoader(filepath)
            raw_documents = loader.load()

            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=300,
                chunk_overlap=100,
                length_function=len,
                is_separator_regex=False,
            )
            chunks = text_splitter.split_documents(raw_documents)
            print ("Document Split Complete",filename)
            # Generate a summary
            full_text = " ".join([chunk.page_content for chunk in chunks])
            summary_text = summarize_text(full_text)

            # Create a summary document
            summary_doc = {"page_content": summary_text, "metadata": {"filename": filename, "type": "summary"}}

            # Index original document chunks
            uuids = [str(uuid4()) for _ in range(len(chunks))]
            vector_store.add_documents(documents=chunks, ids=uuids)

            # Index the summary as a separate document
            summary_id = str(uuid4())
            vector_store.add_documents(documents=[summary_doc], ids=[summary_id])

            print(f"Added file: {filename}")

    # Find deleted files
    # deleted_files = stored_files - current_files
    # for filename in deleted_files:
    #     updates = True
    #     if filename.endswith(".pdf"):
    #         filepath = os.path.join(DATA_PATH, filename)
    #         # Find document IDs associated with the deleted file
    #         results = vector_store._collection.get(
    #             where={"source": filepath}, include=["ids"]
    #         )
    #         if "ids" in results:
    #             ids_to_delete = results["ids"]
    #             vector_store._collection.delete(ids=ids_to_delete)
    #             print(f"Deleted file: {filename}")

    print("-" * 80)
    if updates:
        print("Added files: ", new_files)
        # print("Removed files: ", deleted_files)

    else:
        print("No changes detected.")

def run_periodic_check(interval_seconds = 60):
    """
    Runs the periodic file check in a notebook environment.
    """

    print(f"Checked for file changes....")

    process_files(vector_store, embeddings_model)
    # while True:
    #     process_files(vector_store, embeddings_model)
    #     time.sleep(interval_seconds)
    #     print(f"Checked for file changes. Next check in {interval_seconds} seconds...")



In [ ]:
# To start the periodic check in your notebook, call this function:
run_periodic_check()

# To stop the check, you'll need to interrupt the kernel.

Checked for file changes....
current_files: {'AI Bootcamp Syllabus 2024-1.pdf', '.DS_Store', 'AttentionIsAllyouNeed.pdf'}
stored_files: set()
Document Split Complete AI Bootcamp Syllabus 2024-1.pdf


/var/folders/5p/y9vfdxz132b7tj62wggjh0_m0000gn/T/ipykernel_13382/2873985129.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = summarize_chain.run([text])


AttributeError: 'str' object has no attribute 'page_content'

# Testing the knowledge base

In [1]:
!pip freeze

absl-py==1.4.0
accelerate==1.3.0
aiohappyeyeballs==2.5.0
aiohttp==3.11.13
aiosignal==1.3.2
alabaster==1.0.0
albucore==0.0.23
albumentations==2.0.5
ale-py==0.10.2
altair==5.5.0
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.6.0
arviz==0.20.0
astropy==7.0.1
astropy-iers-data==0.2025.3.3.0.34.45
astunparse==1.6.3
atpublic==4.1.0
attrs==25.1.0
audioread==3.0.1
autograd==1.7.0
babel==2.17.0
backcall==0.2.0
beautifulsoup4==4.13.3
betterproto==2.0.0b6
bigframes==1.38.0
bigquery-magics==0.6.0
bleach==6.2.0
blinker==1.9.0
blis==0.7.11
blosc2==3.2.0
bokeh==3.6.3
Bottleneck==1.4.2
bqplot==0.12.44
branca==0.8.1
CacheControl==0.14.2
cachetools==5.5.2
catalogue==2.0.10
certifi==2025.1.31
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.4.1
chex==0.1.89
clarabel==0.10.0
click==8.1.8
cloudpathlib==0.21.0
cloudpickle==3.1.1
cmake==3.31.6
cmdstanpy==1.2.5
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5
cons==0.4.

In [ ]:
num_results = 5

retriever = vector_store.as_retriever(search_kwargs={'k': num_results})

docs = retriever.invoke("What is syllabus of AI boot camp")
for i, doc in enumerate(docs, 1):
    print(f"Result {i}:")
    print(doc)
    print("-" * 80)
#print((docs[0].metadata["source"], docs[0].metadata["page_label"]))